## Using XULE to extract the presentation structure from an XBRL taxonomy  
This interactive Python code copies the [free, open-source XULE plugin](https://github.com/xbrlus/xule/) into an [Arelle](https://pypi.org/project/arelle-release/) installation. Click the first run button below to get started. This step only needs to be completed **once for the session** unless different versions of Arelle or XULE are required. Click the `Show code` link to review/revise the setup script.

In [ ]:
# @title
import os, shutil, sys, site, platform
print('Please wait while Arelle, XULE and some helper packages are installed. \nA XULE version message and input field appear below when the environment is ready.')

# In this example, Arelle and aniso are required to use XULE - get Arelle release details from GitHub (https://github.com/Arelle/arelle/releases).
# Use %pip -q install git+https://git@github.com/Arelle/arelle.git@master to use Arelle's development release
%pip -q install Arelle-release==2.30.11
%pip -q install aniso8601==9.0.1

# 1) locate Arelle's plugin directory (do not modify this location); remove temp and xuledir if they exist
plugindir = site.getsitepackages()[0] + '/arelle/plugin/'
xuledir = plugindir + 'xule/'
xodeldir = plugindir + 'xodel/'
serializerdir = plugindir + 'serializer/'
SimpleXBRLModeldir = plugindir + 'SimpleXBRLModel/'
temp = plugindir + 'temp/'
tempxule = temp + 'plugin/xule/'
tempxodel = temp + 'plugin/xodel/'
tempserializer = temp + 'plugin/serializer/'
tempSimpleXBRLModel = temp + 'plugin/SimpleXBRLModel/'
if os.path.exists(temp):
        if os.path.isdir(temp):
            os.remove(plugindir + 'semanticHash.py')
            shutil.rmtree(xuledir)
            shutil.rmtree(xodeldir)
            shutil.rmtree(serializerdir)
            shutil.rmtree(SimpleXBRLModeldir)
            shutil.rmtree(temp)
else: ''
os.chdir(plugindir)

# 2) copy a XULE release from GitHub (https://github.com/xbrlus/xule/releases) to plugin directory
!git clone --quiet --depth=1 --branch 30041 --single-branch https://github.com/xbrlus/xule.git temp &> /dev/null
shutil.move(temp + 'plugin/semanticHash.py', plugindir)
shutil.move(tempxule, xuledir)
shutil.move(tempxodel, xodeldir)
shutil.move(tempserializer, serializerdir)
shutil.move(tempSimpleXBRLModel, SimpleXBRLModeldir)

# 3) confirm XULE (change -v to -h and re-run to see help contents for Arelle and XULE)
!arelleCmdLine --plugins 'xule' -v
print("\n\nRun the next cell to set parameters for the XULE expression.\nClick the Show code link to review/revise the parameters script.")

In [ ]:
# @title
def get_user_url(options):
    while True:
        print("\n\nType a number from the list or enter another report or taxonomy URI to generate its presentation hierarchy:")
        for i, option in enumerate(options):
            print(f"{i + 1}. {option}")
        choice = input("> ")
        # Check if the user entered a number from the list
        try:
            choice = int(choice)
            if 1 <= choice <= len(options):
                return options[choice - 1]
        except ValueError:
            pass
        # If the user didn't enter a number, take it as custom input
        return choice

options = ["https://www.sec.gov/Archives/edgar/data/720762/000149315224042532/form10-k.htm",
           "https://www.sec.gov/Archives/edgar/data/700764/000107997324001419/vyey_10q-033124.htm",
           "https://xbrl.fasb.org/us-gaap/2024/entire/us-gaap-entryPoint-std-2024.xsd"]

taxonomy = get_user_url(options)
taxonomy_schema = "taxonomy('" + taxonomy + "')"

print("\n\nSelect the number of the file format for output: ")
data = {
    "1": "spreadsheet",
    "2": "json"
    }

for key, value in data.items():
    print(f"{key}. {value}")

choice = input("> ")

if choice in data:
    format = data[choice]
else:
    print("Invalid number - using spreadsheet option.")
    format = "spreadsheet"

location = input("\n\nEnter a path to the output location or leave blank to use the default value /content/: ") or '/content/'

name = input("\n\nEnter a name for the file or leave blank to use the default value 'taxonomy-hierarchy': ") or 'taxonomy-hierarchy'

#this deletes the prior version of XULE file if one exists
if os.path.exists(name + '.xule'):
    os.remove(name + '.xule')
else: ''

### 1. Define a XULE Expression
The cell below creates a XULE file called taxonomy-hierarchy.xule.

The XULE expression in the file creates an Excel spreadsheet of the presentation hierarchy for a filing (defined in the expression as $schema) and saves it in location `/content/` as the file `taxonomy-hierarchy.xlsx`. This location (as well as references to it in step 2) assumes Colab is the environment, and should be adjusted based on local machine or online service location for storage.

Each tab in the spreadsheet is an array corresponding to a section of the presentation linkbase (ie. the financial statements and its disclosures) The function .agg-to-dict(1) is applied to each different set of 6 characters produced by the list (and printed to the first column) to create spreadsheet tabs.

The columns in each worksheet (1 - 9) include details from the presentation linkbase. To create a visual hierarchy, an `if/else condition` evaluates each concept's navigation-depth and preferred-label and applies blank spaces to the presentation label within the statement or disclosure's strucuture.

The `/** commented  lines to output as .json**/` in the expression creates the arrays in this format (remember to comment out the .xlsx lines)

Click the code cell below then **use 'Run All Below' from Jupyter's Cell menu** to save and run XULE with a single click (NB: after the cell is run once, toggling **%%writefile** to **%load** will display the contents of the sample.xule file in the cell).

In [ ]:
xule_file = '''
constant $schema = %s
constant $filetype = '%s'
constant $filedir = '%s'
constant $filename = '%s'

output-attribute file-location
output-attribute file-content

output taxonomy

$presentation = (navigate parent-child descendants taxonomy $schema
    returns  (
        role-description,
        source-name,
        target-name,
        order,
        navigation-order,
        preferred-label,
        result-order,
        navigation-depth,
        target
    )
)

$updatedlist = filter $presentation returns list(
    $item[1].substring(1,6),
    $item[1],
    $item[2],
    $item[3],
    $item[4],
    $item[5],
    if
      $item[6].text == none '  '.repeat($item[8]) + $item[9].label.text
    else
      '  '.repeat($item[8]) + $item[6].text)

$presentation_updated = $updatedlist.agg-to-dict(1)

if $filetype != 'spreadsheet'
  $presentation_updated.to-json
  file-location $filedir + $filename + '.json' 
else
  $presentation_updated.to-spreadsheet
  file-location $filedir + $filename + '.xlsx'

file-content $rule-value
'''
with open(name + '.xule', mode='w') as file:
    file.write(xule_file % (taxonomy_schema, format, location, name))

### 2. Run XULE against an XBRL report
After defining variables for files and locations, the first Arelle command compiles the XULE into a .zip. After the XULE and compiled .zip files are copied to the same location defined above (from the default location in the XULE plugin directory), the second Arelle command runs the .zip to create the output.

In [ ]:
FILE_NAME = name + '.xule'
ZIP_NAME = name + '.zip'
FILE_LOCATION = location
LOG_LOCATION = location + 'log.xml'
print('This process compiles and runs a XULE .zip for ' + taxonomy + 
'\nto produce its presentation hierarchy. \n\nThe -f argument below is a placeholder report URI that must match the base taxonomy year \nof the report or taxonomy URI input above (ie. 2024 US GAAP).\n\n') 

# compile XULE into .zip 
!arelleCmdLine --plugins "xule" --xule-compile $FILE_NAME \
--xule-rule-set $ZIP_NAME --logFormat="[%(messageCode)s] %(message)s"

# copy XULE and .zip to /content/ directory (a Colab-specific location)
shutil.copy(FILE_NAME, FILE_LOCATION+FILE_NAME)
shutil.copy(ZIP_NAME, FILE_LOCATION+ZIP_NAME)

# run .zip to create output
!arelleCmdLine --plugins "xule|transforms/SEC|validate/EFM|inlineXbrlDocumentSet" \
--xule-rule-set $ZIP_NAME \
-v -f "https://www.sec.gov/Archives/edgar/data/720762/000149315224042532/form10-k.htm" \
--xule-time .000 --xule-debug --noCertificateCheck \
--httpUserAgent "XULE-Arelle (xbrl.us; support@xbrl.us)" --logFile $LOG_LOCATION


The commands in the cell above:
1. uses the XULE plugin to compile the XULE file from step 1 into a .zip file (XULE must be compiled before it will run)
1. copies the source .xule and compiled .zip files to the content folder in the environment (click the folder at left to download), and
1. uses XULE and other plugins to run the .zip against an XBRL report listed in the command (the SEC requires a User Agent to access reports programmatically), create a file according to the XULE expression (.xlsx or .json) and generate a log.xml of the process. 

The XULE expression and Arelle commands define `/content/` as the location for files - XULE, .zip, .xlsx and .xml - so these outputs will appear in the folder at left (default Colab environment) after processing completes. 

If the `$schema` constant in the expression is updated for a 'base' taxonomy (ie. FASB, FERC, IFRS), a report - also known as an instance document - using the same base taxonomy must be used with the last command in the sequence.

(NB: run the first code cell in this notebook with the -h switch instead of -v to get configuration paramters for the command).
